# Ainda em desenvolvimento

In [ ]:
def intersection(l1, l2):
    return list(set(l1) & set(l2))

In [ ]:
lucro_df = df_ativos.loc[(df_ativos['L/P'] != 0.0), ['Data', 'Código', 'L/P']].groupby(['Data', 'Código']).sum()
lucro_df

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def mov_ativo(cod):
    return df_ativos[df_ativos['Código'] == cod]

### Movimentações por ativos

In [ ]:
interact(mov_ativo, cod=df_ativos['Código'].unique());

### Imposto de Renda

- Os dados do imposto de renda devem ser feitos separadamente para cada corretora.
- Os dados para as ações que não foram zeradas estão listadas na carteira.
- Sempre conferir os valores com o site da B3. <- Nem sempre é o mesmo.

In [ ]:
def position(df):

    cods = df['Código'].unique()

    zerada = []
    comprada = []

    for cod in cods:

        qnt = df.loc[df['Código'] == cod, 'Quantidade Atual'].tail(1).values[0]

        if qnt != 0:
            comprada.append(cod)
        else:
            zerada.append(cod)

    return zerada, comprada

In [ ]:
def weighted_mean_for_duplicates(df):
    
    cod_duplicated = df['Código'].value_counts() != 1
    
    cod_unique = [not i for i in cod_duplicated]
    
    new_df = pd.DataFrame()
    
    for cod in cod_duplicated.index[cod_duplicated]:
        
        aux = df[df['Código'] == cod].reset_index(drop=True)
        
        aux.loc[0, 'Preço Atual'] = np.average(aux['Preço Atual'], weights = np.abs(aux['Quantidade de C/V']))
        aux.loc[0, 'Preço de C/V'] = np.average(aux['Preço de C/V'], weights = np.abs(aux['Quantidade de C/V']))
    
        new_df = new_df.append(aux.loc[0, :])
    
    new_df = new_df.append(df.set_index('Código').drop(cod_duplicated.index[cod_duplicated]).reset_index())
    
    new_df = new_df[['Data', 'Código', 'Preço Atual', 'Preço de C/V']].reset_index(drop=True)
    
    new_df['Preço Atual'] = new_df['Preço Atual'].round(3)
    new_df['Preço de C/V'] = new_df['Preço de C/V'].round(3)
    
    new_df['Posição'] = 'Zerada'
    
    return new_df

In [1]:
#### Por enquanto só serve pra ver quais ações foram zeradas

In [ ]:
cols = ['Data', 'Código', 'Preço Atual', 'Preço de C/V', 'Quantidade de C/V']
zerada, comprada = position(df_ativos)

ir_df = weighted_mean_for_duplicates(df_ativos.loc[df_ativos['Código'].isin(zerada), cols]).sort_values('Código')
ir_df